<a href="https://colab.research.google.com/github/BenjaminUy/Predicting-Loan-User-Default-Risk/blob/main/notebooks/Model_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **India Loan Users - Model Development**
Notebook creator: Benjamin Uy

Date created: 6/30/2025

---
Introduction: This is my Jupyter notebook for model development on a dataset of loan customers from India.

The dataset I will use is a modifed and cleaned version of Kaggle user Subham Surana's "Loan Prediction Based on Customer Behavior" (link below).

Link to original dataset: https://www.kaggle.com/datasets/subhamjain/loan-prediction-based-on-customer-behavior?select=Training+Data.csv

Link to modified dataset: https://github.com/BenjaminUy/Predicting-Loan-User-Default-Risk/blob/main/datasets/loan_users_cleaned.csv